In [1]:
import numpy as np
from pyscf import fci
from copy import deepcopy, copy

#define vacuum
#take in operator string
#format: string of indices, bitstring of creation vs annihilation
#return normal-ordered operator string plus contracted terms, according to Wick's theorem



In [109]:
class basicOperator:
    def __init__(self, orbital_, spin_, creation_annihilation_):
        self.orbital = orbital_ #orbital index
        self.spin = spin_ #1 for alpha, 0 for beta
        self.creation_annihilation = creation_annihilation_ #1 for creation, 0 for annihilation

    def apply(self, state):
        neleca, nelecb = state.nelec[0], state.nelec[1]
        if bool(self.creation_annihilation):
            if bool(self.spin):
                state.array = fci.addons.cre_a(state.array, state.norb, state.nelec, self.orbital)
                state.nelec = (neleca + 1, nelecb)
            else:
                state.array = fci.addons.cre_b(state.array, state.norb, state.nelec, self.orbital)
                state.nelec = (neleca, nelecb + 1)
        else:
            if bool(self.spin):
                state.array = fci.addons.des_a(state.array, state.norb, state.nelec, self.orbital)
                state.nelec = (neleca - 1, nelecb)
            else:
                state.array = fci.addons.des_b(state.array, state.norb, state.nelec, self.orbital)
                state.nelec = (neleca, nelecb - 1)

In [ ]:
class operatorSum:
    def __init__(self, operator1_, operator2_):
        self.operator1 = operator1_
        self.operator2 = operator2_

    def apply(self, state):
        state = self.operator1.apply(state) + self.operator2.apply(state)

In [96]:
class operatorString:
    def __init__(self, string_):
        self.string = string_
        #list of indices in the string of second-quantised operators
        self.iS = []
        #bitstring of same length, with 1 corresponding to creation and 0 for annihilation
        self.cOA = []
        #bitstring of same length, with 1 corresponding to alpha and 0 for beta
        self.aOB = []

    #Get indices, alpha or beta, and creation or annihilation from string
    def parse(self):
        c = 0
        while c < len(self.string) - 1:
            self.iS.append(int(self.string[c]))
            if self.string[c+1] == "a":
                    self.aOB.append(1)
            elif self.string[c+1] == "b":
                    self.aOB.append(0)
            if c < len(self.string) - 2 and self.string[c+2] == "+":
                self.cOA.append(1)
                c = c + 3
            else:
                self.cOA.append(0)
                c = c + 2

    #Change string based on new indices, alpha or beta, and creation or annihilation
    def updateString(self):
        self.string = ""
        i = 0
        while i < len(self.iS):
            if bool(self.cOA[i]):
                if bool(self.aOB[i]):
                    self.string = self.string + str(self.iS[i]) + "a+"
                else:
                    self.string = self.string + str(self.iS[i]) + "b+"
            else:
                if bool(self.aOB[i]):
                    self.string = self.string + str(self.iS[i]) + "a"
                else:
                    self.string = self.string + str(self.iS[i]) + "b"
            i = i + 1

    #Apply the string of second-quantised operators to an arbitrary state object
    #Note that nelec does not yet account for the state being completely annihilated
    def apply(self, state):
        i = len(self.iS)
        while(i > 0):
            i = i - 1
            print(i)
            print(self.iS[i])
            print(self.cOA[i])
            print(self.aOB[i])
            neleca, nelecb = state.nelec[0], state.nelec[1]
            if bool(self.cOA[i]):
                if bool(self.aOB[i]):
                    state.array = fci.addons.cre_a(state.array, state.norb, state.nelec, self.iS[i])
                    state.nelec = (neleca + 1, nelecb)
                else:
                    state.array = fci.addons.cre_b(state.array, state.norb, state.nelec, self.iS[i])
                    state.nelec = (neleca, nelecb + 1)
            else:
                if bool(self.aOB[i]):
                    state.array = fci.addons.des_a(state.array, state.norb, state.nelec, self.iS[i])
                    state.nelec = (neleca - 1, nelecb)
                else:
                    state.array = fci.addons.des_b(state.array, state.norb, state.nelec, self.iS[i])
                    state.nelec = (neleca, nelecb - 1)

In [75]:
class state:
    def __init__(self, array_, norb_, nelec_):
        self.array = array_
        self.norb = norb_
        self.nelec = nelec_

    def getArray(self):
        return self.array

    def getNorb(self):
        return self.norb
    
    def getNelec(self):
        return self.nelec

    def setArray(self, array_):
        self.array = array_

    def setNorb(self, norb_):
        self.norb = norb_

    def setNelec(self, nelec_):
        self.nelec = nelec_

In [76]:
ref = state(np.array([[1.,0.,0.],[0.,0.,0.],[0.,0.,0.]]), 3, (2,1))

In [5]:
ref = state(np.array([[1.]]), 1, (1,0))

In [97]:
opStr = operatorString("0a1b+2a+")
opStr.parse()

In [98]:
opStr.iS

[0, 1, 2]

In [99]:
new = deepcopy(ref)
opStr.apply(new)

2
2
1
1
1
1
1
0
0
0
0
1


In [100]:
ref.nelec

(2, 1)

In [101]:
new.nelec

(2, 2)

In [102]:
new.array

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [-1.,  0.,  0.]])

In [103]:
opStr1 = operatorString("0a+0b+1a+")
opStr1.parse()
print(opStr1.string)
print(opStr1.iS)
print(opStr1.aOB)
print(opStr1.cOA)

0a+0b+1a+
[0, 0, 1]
[1, 0, 1]
[1, 1, 1]


In [104]:
opStr2 = copy(opStr1)

In [105]:
type(opStr2.string)

str

In [106]:
opStr2.iS[2] = 2
print(opStr2.string)
print(opStr2.iS)
print(opStr2.aOB)
print(opStr2.cOA)

0a+0b+1a+
[0, 0, 2]
[1, 0, 1]
[1, 1, 1]


In [107]:
opStr2.updateString()

In [108]:
opStr2.string

'0a+0b+2a+'